# LLM Optimization Modelling Experiment

In [156]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [248]:
problem = '''You are in charge of the supply purchasing of a company that produces two kinds of drugs. 
The drugs contain a specific active agent, which is extracted from two different kinds of raw materials that should be purchased on the market. 

The goal is to maximize the total profit obtained from producing the drugs, which means minimizing purchasing costs for buying the raw materials and minimizing operational costs for producing the drugs while maximizing sales. 

You are given a budget for purchasing raw materials and operating the production process of the drugs which cannot be exceeded. 
Additionally, you need to keep the capacity constraints for the production of the drugs in mind as there is only a limited amount of manpower as well as equipment hours available. 
Your company can also only store a limited amount of raw materials. 
Most importantly, the amount of active ingredient extracted from the raw materials you purchase needs to satisfy the required amount of active ingredient in the drugs your company produces. 
Importantly, your experience with the production of the drugs has shown the the amount of active ingredient your process extracts from the two raw materials can vary by up to 0.5% and 2%, respectively. 
During purchasing, you need to make sure that no matter how much the amount of active ingredient varies, the required amount for the drug production is always met. 
'''

## 2. Generate the mathematical model

In [269]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [270]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Drug Production

**Sets and Indices:**

* $i \in \{1,2\}$: Index for raw materials.
* $j \in \{1,2\}$: Index for drugs.

**Parameters:**

* $c_i$: Purchasing cost per unit of raw material $i$ (€/unit).
* $p_j$: Selling price per unit of drug $j$ (€/unit).
* $o_j$: Operational cost per unit of drug $j$ (€/unit).
* $B$: Total budget for purchasing raw materials and production (€).
* $C_j$: Production capacity for drug $j$ (units).
* $S_i$: Storage capacity for raw material $i$ (units).
* $R_{ij}$: Nominal amount of active agent extracted from one unit of raw material $i$ for producing one unit of drug $j$ (units/unit).
* $\delta_{i}$: Maximum deviation of active agent extraction from raw material $i$, where $\delta_1 = 0.005$ and $\delta_2 = 0.02$.
* $D_j$: Required amount of active agent for producing one unit of drug $j$ (units/unit).

**Decision Variables:**

* $x_i$: Amount of raw material $i$ to purchase (units).
* $y_j$: Amount of drug $j$ to produce (units).

**Objective Function:**

Maximize the total profit: 

$$\max \sum_{j=1}^{2} p_j y_j - \sum_{i=1}^{2} c_i x_i - \sum_{j=1}^{2} o_j y_j $$

**Constraints:**

1. **Budget constraint:** The total cost of purchasing raw materials and production cannot exceed the budget.
$$\sum_{i=1}^{2} c_i x_i + \sum_{j=1}^{2} o_j y_j \le B$$

2. **Capacity constraint:** The production of each drug cannot exceed its capacity limit.
$$y_j \le C_j,  \forall j \in \{1,2\}$$

3. **Storage constraint:** The amount of each raw material purchased cannot exceed its storage capacity.
$$x_i \le S_i,  \forall i \in \{1,2\}$$

4. **Active agent requirement constraint:** The amount of active agent extracted from the purchased raw materials must be sufficient to meet the requirement for each drug, considering the maximum deviation in extraction rates.
$$\sum_{i=1}^{2} (R_{ij} - \delta_{i}R_{ij}) x_i  \ge D_j y_j,  \forall j \in \{1,2\}$$

5. **Non-negativity constraint:** The amount of raw materials purchased and drugs produced cannot be negative.
$$x_i \ge 0, \forall i \in \{1,2\}$$
$$y_j \ge 0, \forall j \in \{1,2\}$$

This model incorporates all given information and constraints to determine the optimal purchase amounts of each raw material and production amounts of each drug to maximize profit. 


## 3. Generate the pyomo code

In [271]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [272]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with actual data
c = {1: 50, 2: 60}   # Cost of raw materials
p = {1: 150, 2: 200}  # Selling price of drugs
o = {1: 20, 2: 30}   # Operational cost of drugs
B = 10000              # Total budget
C = {1: 200, 2: 150}  # Production capacity
S = {1: 500, 2: 400}  # Storage capacity
R = {(1, 1): 0.8, (1, 2): 0.6, 
     (2, 1): 0.5, (2, 2): 0.7}  # Active agent extraction rate
delta = {1: 0.005, 2: 0.02}  # Maximum deviation
D = {1: 0.5, 2: 0.6}  # Active agent requirement

# Model Creation
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=[1, 2]) # Raw materials
model.J = pyo.Set(initialize=[1, 2]) # Drugs

# Parameters
model.c = pyo.Param(model.I, initialize=c)
model.p = pyo.Param(model.J, initialize=p)
model.o = pyo.Param(model.J, initialize=o)
model.B = pyo.Param(initialize=B)
model.C = pyo.Param(model.J, initialize=C)
model.S = pyo.Param(model.I, initialize=S)
model.R = pyo.Param(model.I, model.J, initialize=R)
model.delta = pyo.Param(model.I, initialize=delta)
model.D = pyo.Param(model.J, initialize=D)

# Decision Variables
model.x = pyo.Var(model.I, domain=pyo.NonNegativeReals)  # Amount of raw material to purchase
model.y = pyo.Var(model.J, domain=pyo.NonNegativeReals)  # Amount of drug to produce

# Objective Function
def profit(model):
    return sum(model.p[j]*model.y[j] for j in model.J) - sum(model.c[i]*model.x[i] for i in model.I) - sum(model.o[j]*model.y[j] for j in model.J)
model.profit = pyo.Objective(rule=profit, sense=pyo.maximize)

# Constraints
def budget_constraint(model):
    return sum(model.c[i]*model.x[i] for i in model.I) + sum(model.o[j]*model.y[j] for j in model.J) <= model.B
model.budget_constraint = pyo.Constraint(rule=budget_constraint)

def capacity_constraint(model, j):
    return model.y[j] <= model.C[j]
model.capacity_constraint = pyo.Constraint(model.J, rule=capacity_constraint)

def storage_constraint(model, i):
    return model.x[i] <= model.S[i]
model.storage_constraint = pyo.Constraint(model.I, rule=storage_constraint)

def active_agent_constraint(model, j):
    return sum((model.R[i,j] - model.delta[i]*model.R[i,j])*model.x[i] for i in model.I) >= model.D[j]*model.y[j]
model.active_agent_constraint = pyo.Constraint(model.J, rule=active_agent_constraint)

# Solve the model
solver = pyo.SolverFactory('glpk')  # Choose a suitable solver
solver.solve(model)

# Print the results
print("Optimal Solution:")
print("Raw Material Purchases (x):")
for i in model.I:
    print(f"  Raw Material {i}: {pyo.value(model.x[i])}")
print("Drug Production (y):")
for j in model.J:
    print(f"  Drug {j}: {pyo.value(model.y[j])}")
print(f"Total Profit: {pyo.value(model.profit)}")
```


## 4. Input problem data and try running the generated code

In [273]:
import pyomo.environ as pyo

# Sample Data - Replace with actual data
c = {1: 100, 2: 199.90}  # Cost per unit of raw material
p = {1: 6200, 2: 6900}  # Selling price per unit of drug
o = {1: 700, 2: 800}   # Operational cost of drugs
B = 100000              # Total budget
C = {1: 200, 2: 150}  # Production capacity
S = {1: 500, 2: 400}  # Storage capacity
R = {(1, 1): 0.8, (1, 2): 0.6, 
     (2, 1): 0.5, (2, 2): 0.7}  # Active agent extraction rate
delta = {1: 0.005, 2: 0.02}  # Maximum deviation
D = {1: 0.5, 2: 0.6}  # Active agent requirement

# Model Creation
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=[1, 2]) # Raw materials
model.J = pyo.Set(initialize=[1, 2]) # Drugs

# Parameters
model.c = pyo.Param(model.I, initialize=c)
model.p = pyo.Param(model.J, initialize=p)
model.o = pyo.Param(model.J, initialize=o)
model.B = pyo.Param(initialize=B)
model.C = pyo.Param(model.J, initialize=C)
model.S = pyo.Param(model.I, initialize=S)
model.R = pyo.Param(model.I, model.J, initialize=R)
model.delta = pyo.Param(model.I, initialize=delta)
model.D = pyo.Param(model.J, initialize=D)

# Decision Variables
model.x = pyo.Var(model.I, domain=pyo.NonNegativeReals)  # Amount of raw material to purchase
model.y = pyo.Var(model.J, domain=pyo.NonNegativeReals)  # Amount of drug to produce

# Objective Function
def profit(model):
    return sum(model.p[j]*model.y[j] for j in model.J) - sum(model.c[i]*model.x[i] for i in model.I) - sum(model.o[j]*model.y[j] for j in model.J)
model.profit = pyo.Objective(rule=profit, sense=pyo.maximize)

# Constraints
def budget_constraint(model):
    return sum(model.c[i]*model.x[i] for i in model.I) + sum(model.o[j]*model.y[j] for j in model.J) <= model.B
model.budget_constraint = pyo.Constraint(rule=budget_constraint)

def capacity_constraint(model, j):
    return model.y[j] <= model.C[j]
model.capacity_constraint = pyo.Constraint(model.J, rule=capacity_constraint)

def storage_constraint(model, i):
    return model.x[i] <= model.S[i]
model.storage_constraint = pyo.Constraint(model.I, rule=storage_constraint)

def active_agent_constraint(model, j):
    return sum((model.R[i,j] - model.delta[i]*model.R[i,j])*model.x[i] for i in model.I) >= model.D[j]*model.y[j]
model.active_agent_constraint = pyo.Constraint(model.J, rule=active_agent_constraint)

# Solve the model
solver = pyo.SolverFactory('glpk')  # Choose a suitable solver
solver.solve(model)

# Print the results
print("Optimal Solution:")
print("Raw Material Purchases (x):")
for i in model.I:
    print(f"  Raw Material {i}: {pyo.value(model.x[i])}")
print("Drug Production (y):")
for j in model.J:
    print(f"  Drug {j}: {pyo.value(model.y[j])}")
print(f"Total Profit: {pyo.value(model.profit)}")

Optimal Solution:
Raw Material Purchases (x):
  Raw Material 1: 89.5335303071
  Raw Material 2: 0.0
Drug Production (y):
  Drug 1: 142.537380248903
  Drug 2: 89.0858626555645
Total Profit: 29197.779568448357


## 5. Correct the code to verify model viability (optional)

In [265]:
#This is correct code to compare, but with some parameters input manually.
from pyomo.environ import *

# Sample data (replace with actual data)
c = {1: 100, 2: 199.90}  # Cost per unit of raw material
p = {1: 6200, 2: 6900}  # Selling price per unit of drug
o = {1: 700, 2: 800}   # Operational cost per unit of drug
a = {
    (1, 1): 0.01,
    (1, 2): 0.01,
    (2, 1): 0.02,
    (2, 2): 0.02
}  # Nominal amount of active ingredient
alpha = {1: 0.005, 2: 0.02}  # Variability percentage
r = {1: 0.5, 2: 0.6}  # Required active ingredient per unit
B = 100000  # Budget
C = 1000  # Storage capacity #ALTERED
M = 2000  # Available manpower hours
E = 800  # Available equipment hours
m = {1: 90, 2: 100}  # Manpower hours per unit
e = {1: 40, 2: 50}  # Equipment hours per unit

# Model creation
model = ConcreteModel()

# Sets
model.I = Set(initialize=c.keys())  # Raw materials
model.J = Set(initialize=p.keys())  # Drugs

# Parameters
model.c = Param(model.I, initialize=c)
model.p = Param(model.J, initialize=p)
model.o = Param(model.J, initialize=o)
model.a = Param(model.I, model.J, initialize=a)
model.alpha = Param(model.I, initialize=alpha)
model.r = Param(model.J, initialize=r)
model.B = Param(initialize=B)
#model.C = Param(model.I, initialize=C)
model.M = Param(initialize=M)
model.E = Param(initialize=E)
model.m = Param(model.J, initialize=m)
model.e = Param(model.J, initialize=e)

# Decision variables
model.x = Var(model.I, domain=NonNegativeReals)
model.y = Var(model.J, domain=NonNegativeReals)

# Objective function
model.profit = Objective(
    expr=sum((model.p[j] - model.o[j]) * model.y[j] for j in model.J) -
    sum(model.c[i] * model.x[i] for i in model.I),
    sense=maximize
)

# Constraints
model.budget_constraint = Constraint(
    expr=sum(model.c[i] * model.x[i] for i in model.I) +
    sum(model.o[j] * model.y[j] for j in model.J) <= model.B
)
model.storage_constraint = Constraint(expr=sum(model.x[i] for i in model.I)<=C)
model.manpower_constraint = Constraint(expr=sum(model.m[j] * model.y[j] for j in model.J) <= model.M)
model.equipment_constraint = Constraint(expr=sum(model.e[j] * model.y[j] for j in model.J) <= model.E)
model.active_ingredient_constraint = Constraint(expr=(0.01*0.995*model.x[1] + 0.02*0.98*model.x[2] - 0.5*model.y[1] - 0.6* model.y[2]>=0))

# Solve the model
solver = SolverFactory('glpk')  # You can change the solver as needed
results = solver.solve(model)

# Print results (you might want to improve formatting)
print(results)

# Accessing results
print("Optimal profit:", model.profit())
for i in model.I:
    print(f"Quantity of raw material {i} to purchase: {model.x[i]():.2f}")
for j in model.J:
    print(f"Quantity of drug {j} to produce: {model.y[j]():.2f}")


Problem: 
- Name: unknown
  Lower bound: 8294.56683928728
  Upper bound: 8294.56683928728
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 4
  Number of nonzeros: 14
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.03960728645324707
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Optimal profit: 8294.566839287349
Quantity of raw material 1 to purchase: 877.73
Quantity of raw material 2 to purchase: 0.00
Quantity of drug 1 to produce: 17.47
Quantity of drug 2 to produce: 0.00


## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [274]:
print(response.text)

## Mathematical Optimization Model for Drug Production

**Sets and Indices:**

* $i \in \{1,2\}$: Index for raw materials.
* $j \in \{1,2\}$: Index for drugs.

**Parameters:**

* $c_i$: Purchasing cost per unit of raw material $i$ (€/unit).
* $p_j$: Selling price per unit of drug $j$ (€/unit).
* $o_j$: Operational cost per unit of drug $j$ (€/unit).
* $B$: Total budget for purchasing raw materials and production (€).
* $C_j$: Production capacity for drug $j$ (units).
* $S_i$: Storage capacity for raw material $i$ (units).
* $R_{ij}$: Nominal amount of active agent extracted from one unit of raw material $i$ for producing one unit of drug $j$ (units/unit).
* $\delta_{i}$: Maximum deviation of active agent extraction from raw material $i$, where $\delta_1 = 0.005$ and $\delta_2 = 0.02$.
* $D_j$: Required amount of active agent for producing one unit of drug $j$ (units/unit).

**Decision Variables:**

* $x_i$: Amount of raw material $i$ to purchase (units).
* $y_j$: Amount of drug $j$ to 

In [275]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with actual data
c = {1: 50, 2: 60}   # Cost of raw materials
p = {1: 150, 2: 200}  # Selling price of drugs
o = {1: 20, 2: 30}   # Operational cost of drugs
B = 10000              # Total budget
C = {1: 200, 2: 150}  # Production capacity
S = {1: 500, 2: 400}  # Storage capacity
R = {(1, 1): 0.8, (1, 2): 0.6, 
     (2, 1): 0.5, (2, 2): 0.7}  # Active agent extraction rate
delta = {1: 0.005, 2: 0.02}  # Maximum deviation
D = {1: 0.5, 2: 0.6}  # Active agent requirement

# Model Creation
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=[1, 2]) # Raw materials
model.J = pyo.Set(initialize=[1, 2]) # Drugs

# Parameters
model.c = pyo.Param(model.I, initialize=c)
model.p = pyo.Param(model.J, initialize=p)
model.o = pyo.Param(model.J, initialize=o)
model.B = pyo.Param(initialize=B)
model.C = pyo.Param(model.J, initialize=C)
model.S = pyo.Param(model.I, initialize=S)
model.R = pyo.Param(model.I, model.J, initialize